Partiamo con un circuito che è **quasi lumped** (perché abbiamo pezzi di circuito che sono a parametri concentrati, come i transmon, e altri pezzi che sono a parametri distribuiti, come i resonator, ma che noi trattiamo come se fossero a parametri concentrati e.g somma di oscillatori armonici). L'obiettivo è quello di trovare l'Hamiltoniano che regola il funzionamento del circuito. Poiché, in generale, il circuito può essere abbastanza complesso, si cerca una strategia per poter ottenere i parametri necessari alla scrittura dell'Hamiltoniano, come la matrice delle capacità e quella delle induttanze. Se consideriamo le variabili $\Phi$ e $Q$ come gli equivalenti di x e p, avremo che il termine di energia cientica è rappresentato da $\frac{1}{2}Q^TC^{-1}Q$ mentre l'energia potenziale è rappresentato da $\frac{1}{2}\Phi^T L^{-1}\Phi$ (potenziale parabolico) a cui andrà aggiunto il contributo dei termini non lineari (giunzioni). Per fare ciò è possibili effettuare due partizioni diverse del circuito: 1) la partizione in celle 2)la partizione in sottosistemi.
Le celle sono un gruppo di elementi (con elementi intendo letteralmente pezzi di metallo) che sono indipendenti le une dalle altre e che quindi possono essere simulate separatamente. I sottosistemi sono invece i componenti effettivi del circuito (transmon, resonator...). La prima cosa da fare è considerare ogni elemento del circuito come un nodo, tra questi nodi ci saranno quelli di coupling (i nodi toccati solo da elementi dello stesso tipo, e.g solo capacità o solo induttanze) e il ground che andranno eliminati in seguito (perché non aggiungono modi di oscillazione). Simulando le singole celle troviamo le matrici delle capacità $C_n$ e delle induttanze inverse associate $L^{-1}$ (ridotte, quindi senza includere il ground). Le due matrici complessive saranno date dalla somma di tutte le matrici (ovviamente scritte tutte con la stessa dimensione N-1xN-1). Dopo questo passaggio è già possibile scrivere la Lagrangiana come $L=\frac{1}{2}\dot{\Phi}^TC\dot{\Phi}-\frac{1}{2}\Phi^TL^{-1}\Phi-\sum_{j=1}^{J}{\epsilon_j^{nl}}$. Prima di arrivare a scrivere l'Hamiltoniano è necessario effettuare delle trasformazioni in modo da considerare come base dei flussi quella in cui ci sono i flussi delle giunzioni e poi ridurre le matrici. I passaggi vengono descritti con un esempio.

Consideriamo l'esempio sottostante che rappresenta una cella

In [1]:
%load_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

design = designs.DesignPlanar()
gui = MetalGUI(design)
design.overwrite_enabled = True

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket # import del pacchetto Transmon 
# Imposta le dimensioni del chip, così non va ritagliato
design.chips.main.size['size_x'] = '1.5mm' 
design.chips.main.size['size_y'] = '1.5mm' 
design.chips.main
#con dimensioni più piccole ho errore

{'material': 'silicon',
 'layer_start': '0',
 'layer_end': '2048',
 'size': {'center_x': '0.0mm',
  'center_y': '0.0mm',
  'center_z': '0.0mm',
  'size_x': '1.5mm',
  'size_y': '1.5mm',
  'size_z': '-750um',
  'sample_holder_top': '890um',
  'sample_holder_bottom': '1650um'}}

In [2]:
#lista dei parametri che vogliamo utilizzare nel trasmon dati sotto forma di dizionario. Le chiavi sono i nomi dei parametri definiti nella classe
transmon_options=dict(
    orientation='90',
    connection_pads=dict(
        a = dict(loc_W=+1, loc_H=-1, cpw_extend = '25um'),
        b = dict(loc_W=-1, loc_H=-1, cpw_extend = '25um'),
        c = dict(loc_W=+1, loc_H=+1, cpw_extend = '25um')
    ),
    gds_cell_name='FakeJunction_01',
    )
#istanza del transmon
q1=TransmonPocket(design,'Q1', options=transmon_options)

gui.rebuild()  # rebuild the design and plot
gui.autoscale() # resize GUI to see QComponent
gui.zoom_on_components(['Q1']) #Can also gui.zoom_on_components([q1.name])

In [3]:
#export del gds file
#QDesign registers GDS renderer during init of QDesign.
a_gds = design.renderers.gds

#Show the options for GDS
a_gds.options


{'short_segments_to_not_fillet': 'True',
 'check_short_segments_by_scaling_fillet': '2.0',
 'gds_unit': 0.001,
 'ground_plane': 'True',
 'negative_mask': {'main': []},
 'fabricate': 'False',
 'corners': 'circular bend',
 'tolerance': '0.00001',
 'precision': '0.000000001',
 'width_LineString': '10um',
 'path_filename': '../resources/Fake_Junctions.GDS',
 'junction_pad_overlap': '5um',
 'max_points': '199',
 'cheese': {'datatype': '100',
  'shape': '0',
  'cheese_0_x': '25um',
  'cheese_0_y': '25um',
  'cheese_1_radius': '100um',
  'view_in_file': {'main': {1: True}},
  'delta_x': '100um',
  'delta_y': '100um',
  'edge_nocheese': '200um'},
 'no_cheese': {'datatype': '99',
  'buffer': '25um',
  'cap_style': '2',
  'join_style': '2',
  'view_in_file': {'main': {1: True}}},
 'bounding_box_scale_x': '1.2',
 'bounding_box_scale_y': '1.2'}

In [ ]:
#per disabilitare i layer del gds che non ci interessano (tutti tranne ground e top)
a_gds.options['cheese']['view_in_file']['main'][1] = False
a_gds.options['no_cheese']['view_in_file']['main'][1] = False
a_gds.options['buffer']['view_in_file']['main'][1] = False

In [ ]:
#check che abbia cambiato i valori
a_gds.options

In [ ]:
#fa l'export del gds e lo mette nel path indicato
a_gds.export_to_gds('C:/Users/simyz/OneDrive/Desktop/Qcomputing/SuperconductingQubit/prova1.gds')

#Oss. nel gds non c'è la giunzione, non è un problema perché a noi serve per estrarre la matrice delle capacità

Una volta estratto il gds file, si utilizza LayouEditor per gli ultimi ritocchi (semplicemente separare i cpw dal ground) e poi FastCap per estrarre la matrice delle capacità. Al momento tutti gli elementi (ground incluso), sono considerati nodi della rete, quindi la matrice associata sarà 6x6. L'ordine dei nodi è il seguente:
* N0=P0
* N1=P1
* N2=B1
* N3=B2
* N4=B3
* N5=G

Ricordiamo che i componenti $C_{ij}$ della matrice sono calcolati nel seguente modo: $C_{ij}=-C_{ij}$ se $i\neq j$ altrimenti $C_{ij}=\sum_{j=0}^{N-1}{C_{ij}}$. Questo procedimento viene fatto in automatico da FastCap. Dopodiché si può scrivere la matrice di Maxwell, scegliendo un nodo come ground ed eliminando la riga e la colonna associate a quel nodo.

Per quanto riguarda la matrice delle induttanze, la versione in cui non si considerano i componenti non lineari, sarà una matrice con soli zeri, nel momento in cui andiamo ad aggiungere i componenti non lineari avremo il valore dell'induttanza (inverso) della giunzione nella posizione 00,11 e - il valore nelle posizioni 01,10 (questo lo vedi dal fatto che per scrivere l'hamiltoniano in modo tale che ci sia una somma in j delle energie dei componenti non lineari in automatico la matrice deve essere fatta così). Poi, possiamo ridurre le matrici semplicemente eliminando la riga e la colonna corrispondenti al ground.

Iniziamo col definire la prima trasformazione $S_n^{-1}$ che serve per avere come vettori di base i flussi legati alla giunzione $\Phi=S_n^{-1}\Phi_n$. Nel caso in esame, il flusso della giunzione sarà $\Phi_1-\Phi_0$, usiamo questo vettore al posto di $\Phi_1$, tutti gli altri rimarranno invariati.
$$S_n^{-1}=\begin{bmatrix}  
                       1&0&0&0&0&\\ 
                       -1&1&0&0&0&\\ 
                       0&0&1&0&0&\\ 
                       0&0&0&1&0&\\ 
                       0&0&0&0&1& 
                       \end{bmatrix}
$$

In [1]:
#Creazione della matrice di Maxwell. Prende in ingresso la matrice e il nodo da eliminare
import numpy as np
def MaxwellCapacitance(C,ground):
    C_max=np.delete(C,ground, axis=1)
    C_max=np.delete(C_max,ground,axis=0)
    return C_max

#aggiunta dei componenti non lineari delle induttanze
def NonLin(L,nonlinear_component, nodi):
    if len(nonlinear_component)==len(nodi):
        for i in range(len(nodi)):
            coppia_nodi=nodi[i]
            L[coppia_nodi[0],coppia_nodi[0]]+=nonlinear_component[i]
            L[coppia_nodi[1],coppia_nodi[1]]+=nonlinear_component[i]
            L[coppia_nodi[0],coppia_nodi[1]]+=-nonlinear_component[i]
            L[coppia_nodi[1],coppia_nodi[0]]+=-nonlinear_component[i]
    else:
        print("ERRORE:le dimensioni dei due vettori non coincidono")
    return L    

In [2]:
#implementazione della prima trasformazione Sn la funzione ha come parametri di ingresso la dimensione della base dim, e l'insieme
#delle coppie di nodi a cui sono collegati le induttanze non lineari. Cambia la colonna della matrice associata al secondo nodo
#della coppia
import numpy as np
def Sn(dim,coppie_nodi):
    S=np.identity(dim)
    for coppie in coppie_nodi:
        S[coppie[1],coppie[0]]=-1
    S=np.linalg.inv(S) #ne faccio il suo inverso
    return S
    

In [3]:
#test sull'esempio descritto in precedenza
print(Sn(5,[(0,1)]))

[[1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


Con questa trasformazione, abbiamo trasformato la matrice degli inversi delle induttanze, in una matrice diagonale. Questa matrice, però, potrebbe essere ancora singolare. Serve dunque effettuare una seconda trasformazione per andare ad eliminare i vettori del ker che non danno contributo in quanto i nodi a loro associati sono di coupling (tutti i nodi connessi esclusivamente tramite capacità). Nel caso in esame, la $L^{-1}$ avrà un kernel di dimensione 5. Poiché se si considera il sistema composito (dell'articolo sul LOM), i nodi N2, N3, N4 sono connessi ai cpw e quindi anche se al momento non appare, sono connessi a delle induttanze, gli unici nodi "puramente" di coupling è N0. La matrice $S_r$ che contiene i flussi associati ai nodi di coupling sarà una vettore
$$\begin{bmatrix} 1\\ 0\\ 0\\ 0\\ 0\end{bmatrix}$$
La matrice complementare $S_k$ sarà invece una 4x5:
$$\begin{bmatrix} 0&0&0&0\\ 1&0&0&0\\ 0&1&0&0\\ 0&0&1&0\\ 0&0&0&1 \end{bmatrix}$$
In codice:

In [ ]:
#data la dimmensione totale della matrice delle induttanze (o capacità) e dato il suo ker, trova la trasformazione Sr
def Sr(dim,nodi_ker):
    S=np.zeros((dim, len(nodi_ker)))
    j=0
    for i in nodi_ker:
        S[i,j]=1
        j=j+1
    return S

#data la mdimensione totale della matrice delle induttanze (o capacità) e dato il suo ker, trova il complementare di Sr
#si sfrutta il fatto che molte delle colonne di Sk sono le colonne della matrice identità
def Sk(dim,nodi_ker):
    S=np.identity(dim)
    S_k=np.delete(S,nodi_ker,axis=1)
    return S_k
    

Una volta trovate le matrici è possibile effettuare la seconda trasformazione (trasformazione di riduzione), che nel nostro caso sarà anche l'ultima, visto che non abbiamo coupling induttivi. Per fare ciò, basta applicare la formula 7 dell'articolo che viene implementate sotto, sia per la matrice dell'inverso delle induttanze che per quella delle capacità

In [ ]:
import numpy as np

#Data la matrice di trasformazione(riduzione) Sk e la matrice dell'inverso delle induttanze, ritorna la matrice ridotta
def Lk(Sk,L):
    Sk_T=np.transpose(Sk)
    L_k=np.dot(np.dot(Sk_T,L),Sk)
    return L_k

def Ck(Sr,Sk,C):
    Sr_T=np.transpose(Sr)
    Sk_T=np.transpose(Sk)
    A=np.dot(np.dot(Sr_T,C),Sr) #implemento il pezzo SrT*C*Sr
    A=np.linalg.inv(A) #ne faccio il suo inverso
    B=np.dot(np.dot(np.dot(C,Sr),A),np.dot(Sr_T,C)) #finisco di calcolare tutto il pezzo centrale della 7
    B=C-B
    C_k=np.dot(Sk_T,np.dot(B,Sk))
    return C_k

A questo punto, vogliamo arrivare a scrivere l'Hamiltoniano. Per farlo, è necessario definire il "momento coniugato", che nel nostro caso sarà il vettore delle cariche $Q_k=C_k\Phi_k$. (puoi vederlo come  matrice dove ogni colonna corrisponde a un vettore di carica)

In [ ]:
def Qk(Ck,phi):
    Q_k=np.dot(Ck,phi)
    return Q_k

def Hamiltoniano_lin(Qk,Ck,Lk,phik):
    Qk_T=np.transpose(Qk)
    phik_T=np.transpose(phik)
    C1=np.linalg.inv(Ck)
    T=np.dot(Qk_T,np.dot(C1,Qk))/2
    V=np.dot(phik_T,np.dot(L,phik))/2
    H=T+V
    return H



In [ ]:
#estrazione dei parametri del transmon
def E_c (C,component_riga, component_colonna):
    e = 1.60217657e-19  # electron charge
    c=C[component_riga][component_colonna]
    Ec=e**2/(2*c)
    return Ec

def E_j (Lj):
    e = 1.60217657e-19  # electron charge
    h = 6.62606957e-34  # Plank's
    phi0=h/(2*e)
    Ej=(phi0/(2*np.pi))**2
    Ej=Ej/Lj
    return Ej
#anharmonicity=-Ec
Ec=E_c(C_k,0,0)
a=-Ec
#plasma frequency
hbar = 1.0545718E-34  # Plank's reduced
wp=pow(8*Ec*E_j(Lj),0.5)
wp=wp/hbar

#dressed frequency
wq=wp-(Ec/hbar)


Questo è quello che riguarda le celle con i transmon. Per le celle contenenti i resonator, la derivazione dell'Hamiltoniano è più semplice. Sappiamo, infatti, che nel caso di un oscillatore considerato come un insieme di circuiti LC, l'Hamiltoniano è pari a $H=\sum_m{\hbar \omega_m(a_m^{\dagger}a_m+\frac{1}{2})}$ se si considera le sue terminazioni come degli aperti o dei cortocircuiti. Nel nostro caso, però, avremo che una delle due estremità sarà chiusa su di un carico, è dunque necessario calcolarsi come questo carico influisce sulla struttura dell'Hamiltoniano. Applicando le giuste condizioni al contorno è possibile scrivere un'equazione trascendente che permette di calcolare le $\omega_m$ dressed. Quindi l'unico effetto del carico è quello di ridefinire i valori delle frequenze e delle fasi, la forma dell'Hamiltoniano resterà la stessa. 
Di seguito si prova a risolvere l'equazione 
$$\omega_m\frac{L}{v_p}+arctg(\frac{\omega_m}{\omega_L})=m\pi+b\frac{\pi}{2}$$

In [23]:
import sympy as sp
import numpy as np
l=5E-12 #inserire valore dell'induttanza per unità di lunghezza
c=4E-9 #inserire valore della capacità per unità di lunghezza
CL=1E-9 #inserire il valore della capacità di carico
L=5e-3#inserire lunghezza linea
M=3 #inserire numero massimo di  nodi da considerare
vp=1/np.sqrt(l*c)
b=0 #1 o 0 a seconda della condizione a contorno
w=1/(l*CL*vp)
x = sp.symbols('x')
#risolvo l'equazione
omega=[]
for i in range(M):
    omega.append(sp.nsolve(x*(L/vp)+ sp.atan(x/w)-i*np.pi-b*(np.pi/2),x, (0,1000)))
    print(omega[i])
    


0
3.70240244847131e+21
1.11072073453979e+22
